# Configuration

In [1]:
%load_ext literary.notebook

In [2]:
from functools import lru_cache
from pathlib import Path

In [3]:
CONFIG_FILE_STEM = "literary_config"
CONFIG_FILE_NAMES = (
    f"{CONFIG_FILE_STEM}.json",
    f"{CONFIG_FILE_STEM}.py",
)

In [4]:
@lru_cache()
def find_config_path(*search_paths) -> Path:
    """Find the filepath of the Literary project configuration.

    :param search_paths: starting search paths
    :return:
    """
    visited = set()

    for search_path in search_paths:
        for path in (search_path, *search_path.parents):
            # Avoid re-visiting paths
            if path in visited:
                break
            visited.add(path)

            # Look for any config file
            for name in CONFIG_FILE_NAMES:
                if (path / name).exists():
                    return path

    raise FileNotFoundError("Couldn't find config file")